<a href="https://colab.research.google.com/github/easytoday/RCNN-region-CNN/blob/main/R_CNN(ultime6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- INSTALLATION DE PYCOCOTOOLS ---
print("--- Installation de pycocotools ---")
!pip install cython
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Test de l'installation
try:
    from pycocotools.coco import COCO
    print("pycocotools installé avec succès.")
except ImportError:
    print("ERREUR: pycocotools n'a pas pu être importé. Vérifiez l'installation.")
    import sys; sys.exit("Impossible de continuer sans pycocotools.")


# --- TÉLÉCHARGEMENT DE MS COCO VAL2017 ---
import os
import zipfile

COCO_DATA_DIR = '/content/coco_dataset'
os.makedirs(COCO_DATA_DIR, exist_ok=True)

COCO_VAL_IMAGES_URL = 'http://images.cocodataset.org/zips/val2017.zip'
COCO_VAL_ANNOTATIONS_URL = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'

print(f"\n--- Téléchargement des images de validation COCO 2017 dans {COCO_DATA_DIR}/images/val2017 ---")
!wget -c {COCO_VAL_IMAGES_URL} -P {COCO_DATA_DIR}
print(f"--- Téléchargement des annotations COCO 2017 dans {COCO_DATA_DIR}/annotations ---")
!wget -c {COCO_VAL_ANNOTATIONS_URL} -P {COCO_DATA_DIR}

print("\n--- Décompression des fichiers COCO ---")
with zipfile.ZipFile(os.path.join(COCO_DATA_DIR, 'val2017.zip'), 'r') as zip_ref:
    zip_ref.extractall(os.path.join(COCO_DATA_DIR, 'images'))
    print("val2017.zip décompressé.")

with zipfile.ZipFile(os.path.join(COCO_DATA_DIR, 'annotations_trainval2017.zip'), 'r') as zip_ref:
    zip_ref.extractall(COCO_DATA_DIR)
    print("annotations_trainval2017.zip décompressé.")

print("\n--- Téléchargement et décompression COCO terminés. ---")

# chemins pour qu'ils soient accessibles
COCO_IMAGES_VAL_PATH = os.path.join(COCO_DATA_DIR, 'images', 'val2017')
COCO_ANNOTATIONS_VAL_PATH = os.path.join(COCO_DATA_DIR, 'annotations', 'instances_val2017.json')

--- Installation de pycocotools ---
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-nsuciz2h
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-nsuciz2h
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp311-cp311-linux_x86_64.whl size=395986 sha256=12748e9d12d12c4ef9e3acc3ae24aefcf6fb37a07085acc36c403dd719f4a0c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-nexwfxsc/wheels/6d/69/75/358c50a37672dfda8d74ba3b30ec49fb75d52f7c081886d503
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.9
    Uninstalling pycocotools-2.0.9:
      Successfully uninstalled pycocotools-2.0.9
pycocotools installé avec succès.

--- Téléchargement des images de validation COCO 2017 dans /content/coco_

In [ ]:
import os
import shutil
import requests
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import numpy as np
import cv2
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler # Ajouté pour la normalisation SVM
import xml.etree.ElementTree as ET
from collections import defaultdict
import random
import sys
from datetime import datetime
import time
import json
from pycocotools.coco import COCO

# --- 0. Configuration Initiale et Montage de Google Drive ---
print("--- Initialisation du pipeline R-CNN ---")

start_time_global = datetime.now()
timestamp = start_time_global.strftime('%Y%m%d_%H%M%S')
print(f"Heure de début de l'exécution : {start_time_global.strftime('%Y-%m-%d %H:%M:%S')}")

report_content_list = []

REPORT_FOLDER_DRIVE = '/content/drive/MyDrive/RCP209/Rapports_RCNN'
REPORT_FOLDER_LOCAL = './rapports_locaux'
os.makedirs(REPORT_FOLDER_LOCAL, exist_ok=True)
log_report_local_path = os.path.join(REPORT_FOLDER_LOCAL, f'rapport_rcnn_{timestamp}_local_backup.md')
report_file_path = None

def log_report(message, level="INFO", to_console=True):
    current_time = datetime.now().strftime('[%H:%M:%S]')
    log_line = f"[{level}] {current_time} {message}\n"
    report_content_list.append(log_line)
    if to_console:
        print(log_line.strip())

def save_report_to_file():
    global report_file_path, report_content_list, log_report_local_path
    if report_file_path is None:
        report_file_path = log_report_local_path
    try:
        with open(report_file_path, 'w', encoding='utf-8') as f:
            f.writelines(report_content_list)
    except Exception as e:
        try:
            with open(log_report_local_path, 'w', encoding='utf-8') as f:
                f.writelines(report_content_list)
        except Exception as e_local:
            pass

log_report("Tentative de montage de Google Drive...", level="SETUP")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.makedirs(REPORT_FOLDER_DRIVE, exist_ok=True)
    report_file_path = os.path.join(REPORT_FOLDER_DRIVE, f'rapport_rcnn_{timestamp}.md')
    log_report(f"Google Drive monté avec succès. Les rapports seront enregistrés dans : {REPORT_FOLDER_DRIVE}", level="SETUP")
except Exception as e:
    log_report(f"ERREUR: Impossible de monter Google Drive ou de créer le dossier de rapports : {e}", level="ERROR")
    log_report(f"Les rapports seront uniquement sauvegardés localement dans : {REPORT_FOLDER_LOCAL}", level="WARNING")
    report_file_path = log_report_local_path

report_content_list.append(f"# Rapport d'Exécution R-CNN - {timestamp}\n\n")
report_content_list.append(f"## Résumé de l'Expérience\n")
report_content_list.append(f"* Date et Heure de début de l'exécution : {start_time_global.strftime('%Y-%m-%d %H:%M:%S')}\n")

log_report("\n--- Vérification de l'environnement ---", to_console=True)
report_content_list.append("\n---\n\n## Configuration de l'Environnement\n")

log_report(f"PyTorch version: {torch.__version__}", level="INFO")
report_content_list.append(f"* Version PyTorch : `{torch.__version__}`\n")
log_report(f"Torchvision version: {torchvision.__version__}", level="INFO")
report_content_list.append(f"* Version Torchvision : `{torchvision.__version__}`\n")
log_report(f"OpenCV version: {cv2.__version__}", level="INFO")
report_content_list.append(f"* Version OpenCV : `{cv2.__version__}`\n")
log_report(f"NumPy version: {np.__version__}", level="INFO")
report_content_list.append(f"* Version NumPy : `{np.__version__}`\n")

from sklearn import __version__ as sklearn_version
log_report(f"Scikit-learn version: {sklearn_version}", level="INFO")
report_content_list.append(f"* Version Scikit-learn : `{sklearn_version}`\n")

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    device_properties = torch.cuda.get_device_properties(0)
    total_memory_gb = device_properties.total_memory / (1024**3)
    log_report(f"Un GPU est disponible : {device_name} avec {total_memory_gb:.2f} GB de mémoire.", level="INFO")
    report_content_list.append(f"* GPU Utilisé : `{device_name}`\n")
    report_content_list.append(f"  * Mémoire totale GPU : `{total_memory_gb:.2f} GB`\n")
    report_content_list.append("Le GPU sera utilisé automatiquement pour les calculs intensifs.\n")
else:
    log_report("Aucun GPU trouvé. Toutes les opérations s'exécuteront sur CPU (plus lent voire impossible).", level="WARNING")
    report_content_list.append("* GPU Utilisé : `N/A (CPU seulement)`\n")
    report_content_list.append("Vérifiez votre type de runtime (Exécution > Changer le type d'exécution).\n")

# --- DÉFINITIONS GLOBALES DE CLASSES ET MAPPINGS ---
PASCAL_VOC_CLASSES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]
NUM_CLASSES_VOC = len(PASCAL_VOC_CLASSES)
CNN_CLASSES = PASCAL_VOC_CLASSES + ["background"]

COCO_TO_VOC_CLASSES = {
    'person': 'person', 'bicycle': 'bicycle', 'car': 'car', 'motorcycle': 'motorbike', 'airplane': 'aeroplane',
    'bus': 'bus', 'train': 'train', 'truck': 'car',
    'boat': 'boat', 'traffic light': None, 'fire hydrant': None,
    'stop sign': None, 'parking meter': None, 'bench': None, 'bird': 'bird', 'cat': 'cat', 'dog': 'dog',
    'horse': 'horse', 'sheep': 'sheep', 'cow': 'cow', 'elephant': None, 'bear': None, 'zebra': None,
    'giraffe': None, 'backpack': None, 'umbrella': None, 'handbag': None, 'tie': None, 'suitcase': None,
    'frisbee': None, 'skis': None, 'snowboard': None, 'sports ball': None, 'kite': None, 'baseball bat': None,
    'baseball glove': None, 'skateboard': None, 'surfboard': None, 'tennis racket': None, 'bottle': 'bottle',
    'wine glass': None, 'cup': None, 'fork': None, 'knife': None, 'spoon': None, 'bowl': None, 'banana': None,
    'apple': None, 'sandwich': None, 'orange': None, 'broccoli': None, 'carrot': None, 'hot dog': None,
    'pizza': None, 'donut': None, 'cake': None, 'chair': 'chair', 'couch': 'sofa',
    'potted plant': 'pottedplant', 'bed': None, 'dining table': 'diningtable', 'toilet': None, 'tv': 'tvmonitor',
    'laptop': None, 'mouse': None, 'remote': None, 'keyboard': None, 'cell phone': None, 'microwave': None,
    'oven': None, 'toaster': None, 'sink': None, 'refrigerator': None, 'book': None, 'clock': None,
    'vase': None, 'scissors': None, 'teddy bear': None, 'hair drier': None, 'toothbrush': None
}

# --- DÉFINITION DES CLASSES DATASET ---
class PascalVOCDataset(Dataset):
    def __init__(self, root_dir, year='2007', image_set='trainval', transform=None):
        self.voc_path = root_dir
        self.year = year
        self.image_set = image_set
        self.transform = transform

        self.image_dir = os.path.join(self.voc_path, 'JPEGImages')
        self.annotation_dir = os.path.join(self.voc_path, 'Annotations')
        self.image_set_path = os.path.join(self.voc_path, 'ImageSets', 'Main', image_set + '.txt')

        self.image_paths = []
        self.annotations_data = {}

        log_report(f"Préparation du chargement du dataset PASCAL VOC {year} {image_set} depuis {self.voc_path}...", level="INFO")

        if not os.path.exists(self.image_set_path):
            log_report(f"ATTENTION : Fichier ImageSet non trouvé à {self.image_set_path}.", level="ERROR")
            sys.exit("Fichier ImageSet critique manquant. Arrêt du script.")

        with open(self.image_set_path, 'r') as f:
            image_ids = [line.strip() for line in f]

        for img_id in image_ids:
            img_path = os.path.join(self.image_dir, f'{img_id}.jpg')
            ann_path = os.path.join(self.annotation_dir, f'{img_id}.xml')

            if os.path.exists(img_path) and os.path.exists(ann_path):
                objs = self._parse_annotation_file(ann_path)
                if objs:
                    self.image_paths.append(img_path)
                    self.annotations_data[img_path] = objs

        log_report(f"Structure du Dataset PascalVOCDataset initialisée. Prêt à charger {len(self.image_paths)} images.", level="INFO")

    def _parse_annotation_file(self, xml_file_path):
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        objects = []
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in PASCAL_VOC_CLASSES:
                continue
            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text) - 1
            ymin = float(bbox.find('ymin').text) - 1
            xmax = float(bbox.find('xmax').text) - 1
            ymax = float(bbox.find('ymax').text) - 1

            xmin = max(0.0, xmin)
            ymin = max(0.0, ymin)
            xmax = max(xmin + 1, xmax)
            ymax = max(ymin + 1, ymax)

            objects.append({
                'class_name': class_name,
                'bbox': [xmin, ymin, xmax, ymax]
            })
        return objects

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        annotations = self.annotations_data.get(img_path, [])

        # Le transform sera appliqué sur les régions pour le fine-tuning
        # Mais ici on renvoie l'image PIL pour selective search
        # Pour un transform sur l'image entière faire:
        # if self.transform:
        #     image = self.transform(image)

        return image, annotations, img_path

class COCODataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.coco = COCO(annotation_file)
        self.ids = list(self.coco.imgs.keys())

        self.coco_cats = self.coco.loadCats(self.coco.getCatIds())
        self.coco_id_to_name = {cat['id']: cat['name'] for cat in self.coco_cats}
        self.voc_class_to_idx = {cls_name: i for i, cls_name in enumerate(PASCAL_VOC_CLASSES)}

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        img_id = self.ids[index]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root_dir, img_info['file_name'])

        image = Image.open(img_path).convert('RGB')

        ann_ids = self.coco.getAnnIds(imgIds=img_id, iscrowd=False)
        anns = self.coco.loadAnns(ann_ids)

        boxes = []
        labels = []

        for ann in anns:
            x, y, w, h = ann['bbox']
            bbox = [x, y, x + w, y + h]

            coco_cat_name = self.coco_id_to_name[ann['category_id']]
            voc_class_name = COCO_TO_VOC_CLASSES.get(coco_cat_name)

            if voc_class_name and voc_class_name in PASCAL_VOC_CLASSES:
                boxes.append(bbox)
                labels.append(self.voc_class_to_idx[voc_class_name])

        if len(boxes) > 0:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.long)
        else:
            boxes = torch.empty((0, 4), dtype=torch.float32)
            labels = torch.empty((0,), dtype=torch.long)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id])
        }

        # Le transform sera appliqué plus tard sur les régions d'intérêt, pas sur l'image complète ici
        # Pour le faire sur l'image compléte faire:
        # if self.transform:
        #     image = self.transform(image)

        return image, target, img_path

# --- Fonctions utilitaires ---
def custom_collate_fn(batch):
    if len(batch) == 1:
        return batch[0][0], batch[0][1], batch[0][2]
    else:
        raise ValueError("custom_collate_fn attend un batch de taille 1 pour cette configuration.")

def get_selective_search_proposals(image_path):
    img = cv2.imread(image_path)
    if img is None:
        log_report(f"Avertissement: Impossible de lire l'image {image_path}. Retourne des propositions vides.", level="WARNING")
        return []
    try:
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    except AttributeError:
        log_report("Erreur : cv2.ximgproc.segmentation.createSelectiveSearchSegmentation n'est pas disponible.", level="ERROR")
        sys.exit("Module OpenCV ximgproc manquant. Arrêt du script.")
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    proposals_xyxy = []
    for (x, y, w, h) in rects:
        if w > 0 and h > 0:
            proposals_xyxy.append([x, y, x + w, y + h])
    return proposals_xyxy[:2000]

def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    inter_width = max(0, xB - xA)
    inter_height = max(0, yB - yA)
    inter_area = inter_width * inter_height
    boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union_area = float(boxA_area + boxB_area - inter_area)
    if union_area == 0:
        return 0.0
    return inter_area / union_area

def warp_region(image_pil, bbox, target_size=(227, 227), context_pad=16):
    x_min, y_min, x_max, y_max = bbox
    img_width, img_height = image_pil.size

    x_min_padded = max(0, int(x_min - context_pad))
    y_min_padded = max(0, int(y_min - context_pad))
    x_max_padded = min(img_width, int(x_max + context_pad))
    y_max_padded = min(img_height, int(y_max + context_pad))

    if x_max_padded <= x_min_padded or y_max_padded <= y_min_padded:
        return Image.new('RGB', target_size, (0, 0, 0))

    region = image_pil.crop((x_min_padded, y_min_padded, x_max_padded, y_max_padded))
    warped_region = region.resize(target_size, Image.BILINEAR)
    return warped_region

# --- Fonctions CNN / SVM / Détection ---
def fine_tune_cnn(cnn_model, train_dataset, num_epochs=5, lr=0.001, batch_size=32, iou_threshold=0.5, image_transform=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model.to(device)
    cnn_model.train()

    num_ftrs = cnn_model.classifier[6].in_features
    cnn_model.classifier[6] = nn.Linear(num_ftrs, NUM_CLASSES_VOC + 1).to(device)

    optimizer = optim.SGD(cnn_model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x[0])

    log_report(f"Démarrage du fine-tuning du CNN pour {num_epochs} époques...", level="INFO")
    for epoch in range(num_epochs):
        epoch_loss = 0
        correct_predictions = 0
        total_samples = 0
        for i, (image_pil, annotations, img_path) in enumerate(data_loader):
            if not isinstance(image_pil, Image.Image): # Si le dataset applique déjà le transform
                # reconvertir si c'est un tenseur
                if isinstance(image_pil, torch.Tensor):
                    image_pil = transforms.ToPILImage()(image_pil.cpu())
                else:
                    log_report(f"Type d'image inattendu: {type(image_pil)}. Skipping image.", level="WARNING")
                    continue

            proposals = get_selective_search_proposals(img_path)
            if not proposals:
                continue

            positive_regions = []
            negative_regions = []

            for prop_bbox in proposals:
                max_iou = 0.0
                best_gt_class_idx = -1

                for gt_obj in annotations:
                    iou = calculate_iou(prop_bbox, gt_obj['bbox'])
                    if iou > max_iou:
                        max_iou = iou
                        if gt_obj['class_name'] in PASCAL_VOC_CLASSES:
                            best_gt_class_idx = PASCAL_VOC_CLASSES.index(gt_obj['class_name'])

                if max_iou >= iou_threshold and best_gt_class_idx != -1:
                    positive_regions.append((prop_bbox, best_gt_class_idx))
                elif max_iou < 0.2:
                    negative_regions.append(prop_bbox)

            regions_for_cnn = []
            labels_for_cnn = []

            for bbox, class_idx in positive_regions:
                regions_for_cnn.append(warp_region(image_pil, bbox))
                labels_for_cnn.append(class_idx)

            num_negative_to_add = min(len(negative_regions), len(positive_regions) * 3 if positive_regions else 50)
            if negative_regions:
                random.shuffle(negative_regions)
                for bbox in negative_regions[:num_negative_to_add]:
                    regions_for_cnn.append(warp_region(image_pil, bbox))
                    labels_for_cnn.append(NUM_CLASSES_VOC)

            if not regions_for_cnn:
                continue

            processed_regions = [image_transform(region).to(device) for region in regions_for_cnn]
            labels = torch.tensor(labels_for_cnn, dtype=torch.long).to(device)

            if not processed_regions:
                continue
            batch_regions = torch.stack(processed_regions)

            optimizer.zero_grad()
            outputs = cnn_model(batch_regions)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * batch_regions.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        avg_loss = epoch_loss / total_samples if total_samples > 0 else 0
        accuracy = correct_predictions / total_samples if total_samples > 0 else 0
        log_report(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}", level="INFO")

    log_report("Fine-tuning du CNN terminé.", level="INFO")
    return cnn_model

def extract_features(cnn_model, image_pil, proposals, image_transform):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model.to(device)
    cnn_model.eval()

    features_list = []

    if not proposals:
        return np.array([])

    processed_regions = [warp_region(image_pil, bbox) for bbox in proposals]
    transformed_regions = [image_transform(region).to(device) for region in processed_regions]

    if not transformed_regions:
        return np.array([])

    regions_batch = torch.stack(transformed_regions)

    with torch.no_grad():
        x = cnn_model.features(regions_batch)
        x = cnn_model.avgpool(x)
        x = torch.flatten(x, 1)
        for i, layer in enumerate(cnn_model.classifier):
            x = layer(x)
            if i == 5:
                break

        features_list.append(x.cpu().numpy())

    if features_list:
        return np.vstack(features_list)
    return np.array([])

def train_svms(train_dataset, fine_tuned_cnn_model, iou_threshold=0.5, image_transform=None):   #etait à 0.5 on change à 0.3 revient à 0.5
    log_report("Démarrage de l'entraînement des SVMs...", level="INFO")
    trained_svms = {}

    all_class_features = defaultdict(lambda: {'features': [], 'labels': []})

    data_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: x[0])

    for i, (image_pil, annotations, img_path) in enumerate(data_loader):
        if not isinstance(image_pil, Image.Image): # Si le dataset applique déjà le transform
            if isinstance(image_pil, torch.Tensor):
                image_pil = transforms.ToPILImage()(image_pil.cpu())
            else:
                log_report(f"Type d'image inattendu: {type(image_pil)}. Skipping image.", level="WARNING")
                continue

        proposals = get_selective_search_proposals(img_path)

        if not proposals:
            continue

        all_proposal_features = extract_features(fine_tuned_cnn_model, image_pil, proposals, image_transform)
        if all_proposal_features.size == 0:
            continue

        if len(proposals) != len(all_proposal_features):
            log_report(f"Disparité propositions/features pour {img_path}. Ignoré.", level="WARNING")
            continue

        for idx, prop_bbox in enumerate(proposals):
            feature = all_proposal_features[idx]
            max_iou = 0.0
            best_gt_class_name = None

            for gt_obj in annotations:
                iou = calculate_iou(prop_bbox, gt_obj['bbox'])
                if iou > max_iou:
                    max_iou = iou
                    best_gt_class_name = gt_obj['class_name']

            if max_iou >= iou_threshold and best_gt_class_name in PASCAL_VOC_CLASSES:
                for class_name in PASCAL_VOC_CLASSES:
                    if class_name == best_gt_class_name:
                        all_class_features[class_name]['features'].append(feature)
                        all_class_features[class_name]['labels'].append(1)
                    else:
                        all_class_features[class_name]['features'].append(feature)
                        all_class_features[class_name]['labels'].append(0)

            elif max_iou < 0.2:
                for class_name in PASCAL_VOC_CLASSES:
                    all_class_features[class_name]['features'].append(feature)
                    all_class_features[class_name]['labels'].append(0)

    for class_name in PASCAL_VOC_CLASSES:
        log_report(f"Entraînement du SVM pour la classe '{class_name}'...", level="INFO")
        #features = np.array(all_class_features[class_name]['features'])
        #labels = np.array(all_class_features[class_name]['labels'])

        # S'assurer qu'il y a des échantillons positifs et négatifs
        #if len(features) == 0 or np.sum(labels) == 0 or np.sum(labels == 0) == 0:
        #    log_report(f"Pas assez d'échantillons positifs/négatifs pour entraîner le SVM pour '{class_name}'. Skip.", level="WARNING")
        #    continue

        positive_features_list = [f for f, l in zip(all_class_features[class_name]['features'], all_class_features[class_name]['labels']) if l == 1]
        negative_features_list = [f for f, l in zip(all_class_features[class_name]['features'], all_class_features[class_name]['labels']) if l == 0]

        # Convertir les listes en tableaux numpy
        positive_features = np.array(positive_features_list) if positive_features_list else np.empty((0, all_class_features[class_name]['features'][0].shape[0]))
        negative_features = np.array(negative_features_list) if negative_features_list else np.empty((0, all_class_features[class_name]['features'][0].shape[0]))

        # --- Logique d'échantillonnage des négatives ---
        num_pos = len(positive_features)
        num_neg = len(negative_features)

        # ratio ou un nombre maximum de négatifs à inclure
        # -> Une bonne pratique est de limiter les négatives à 3 fois le nombre de positives,
        # ou à un maximum absolu pour les très grands datasets.
        MAX_NEGATIVES_RATIO = 10 # 5 puis 10 et maintenat 15s
        MAX_NEGATIVES_ABSOLUTE = 100000 # Vous pouvez ajuster cette valeur (ex: 50000, 200000)

        negative_features_sampled = np.empty((0, positive_features.shape[1])) # Initialisation avec la bonne dimension

        if num_neg > 0:
            # Calculer le nombre cible de négatives à inclure
            target_negatives_count = min(num_neg, int(num_pos * MAX_NEGATIVES_RATIO), MAX_NEGATIVES_ABSOLUTE)

            if num_neg > target_negatives_count:
                # Échantillonnage aléatoire des indices
                neg_indices = np.random.choice(num_neg, target_negatives_count, replace=False)
                negative_features_sampled = negative_features[neg_indices]
                log_report(f"Échantillonnage de {len(negative_features_sampled)} négatifs sur {num_neg} pour la classe '{class_name}'.", level="INFO")
            else:
                negative_features_sampled = negative_features
        else:
            log_report(f"Aucun échantillon négatif pour la classe '{class_name}'.", level="WARNING")


        # Combiner les échantillons positifs et les négatifs échantillonnés
        features = np.vstack((positive_features, negative_features_sampled))
        labels = np.hstack((np.ones(len(positive_features)), np.zeros(len(negative_features_sampled))))

        # S'assurer qu'il y a des échantillons pour l'entraînement du SVM après échantillonnage
        if len(features) == 0 or np.sum(labels) == 0 or np.sum(labels == 0) == 0:
            log_report(f"Pas assez d'échantillons positifs/négatifs pour entraîner le SVM pour '{class_name}' après échantillonnage. Skip.", level="WARNING")
            continue

        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)

        svm = LinearSVC(C=0.0005, random_state=42, max_iter=2000) # C was 0.001 puis 0.0001 et maintenant 0.01
        svm.fit(features_scaled, labels)
        trained_svms[class_name] = {'svm': svm, 'scaler': scaler}
        log_report(f"SVM pour '{class_name}' entraîné. Score de classification (accuracy): {svm.score(features_scaled, labels):.4f}", level="INFO")

    log_report("Entraînement des SVMs terminé.", level="INFO")
    return trained_svms

def detect_objects(image_path, fine_tuned_cnn_model, trained_svms, iou_nms_threshold=0.3, image_transform=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    fine_tuned_cnn_model.to(device)
    fine_tuned_cnn_model.eval()

    image_pil = Image.open(image_path).convert("RGB")
    proposals = get_selective_search_proposals(image_path)

    if not proposals:
        return []

    all_proposal_features = extract_features(fine_tuned_cnn_model, image_pil, proposals, image_transform)

    if all_proposal_features.size == 0:
        return []

    detections_by_class = defaultdict(list)

    for class_name in PASCAL_VOC_CLASSES:
        if class_name not in trained_svms:
            continue

        svm_data = trained_svms[class_name]
        svm = svm_data['svm']
        scaler = svm_data['scaler']

        features_scaled = scaler.transform(all_proposal_features)
        scores = svm.decision_function(features_scaled)

        for i, score in enumerate(scores):
            bbox = proposals[i]
            detections_by_class[class_name].append((score, bbox))

    final_detections = []

    for class_name, detections in detections_by_class.items():
        if not detections:
            continue

        detections.sort(key=lambda x: x[0], reverse=True)

        boxes = [d[1] for d in detections]
        scores = [d[0] for d in detections]

        boxes_tensor = torch.tensor(boxes, dtype=torch.float32).to(device)
        scores_tensor = torch.tensor(scores, dtype=torch.float32).to(device)

        indices = torchvision.ops.nms(boxes_tensor, scores_tensor, iou_nms_threshold)

        for idx in indices:
            score = scores[idx.item()]
            bbox = boxes[idx.item()]
            final_detections.append((class_name, score, bbox))

    final_detections.sort(key=lambda x: x[1], reverse=True)
    return final_detections

def evaluate_detector(test_dataset, fine_tuned_cnn_model, trained_svms, iou_threshold_for_tp=0.5, iou_nms_threshold=0.3, image_transform=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    fine_tuned_cnn_model.to(device)
    fine_tuned_cnn_model.eval()

    all_gt_counts_by_class = defaultdict(int)
    all_detections = defaultdict(list)

    actual_test_dataset = test_dataset.dataset if isinstance(test_dataset, Subset) else test_dataset
    is_coco_dataset = isinstance(actual_test_dataset, COCODataset)

    log_report(f"Type de dataset d'évaluation détecté : {'MS COCO' if is_coco_dataset else 'PASCAL VOC'}", level="INFO")

    data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn)

    for image_idx, (image_pil, annotations_or_target, img_path) in enumerate(data_loader):
        current_image_gt_boxes = defaultdict(list)

        if is_coco_dataset:
            coco_target = annotations_or_target
            if coco_target['boxes'].numel() > 0:
                for bbox_tensor, label_tensor in zip(coco_target['boxes'], coco_target['labels']):
                    class_name = PASCAL_VOC_CLASSES[label_tensor.item()]
                    bbox = bbox_tensor.tolist()
                    current_image_gt_boxes[class_name].append(bbox)
                    all_gt_counts_by_class[class_name] += 1
        else: # Pascal VOC Dataset
            pascal_voc_annotations = annotations_or_target
            for gt_obj in pascal_voc_annotations:
                class_name = gt_obj['class_name']
                bbox = gt_obj['bbox']
                current_image_gt_boxes[class_name].append(bbox)
                all_gt_counts_by_class[class_name] += 1

        detected_objects_in_image = detect_objects(img_path, fine_tuned_cnn_model, trained_svms, iou_nms_threshold, image_transform)

        detections_for_image_by_class = defaultdict(list)

        for det_class_name, det_score, det_bbox in detected_objects_in_image:
            detections_for_image_by_class[det_class_name].append({
                'bbox': det_bbox,
                'score': det_score,
                'tp': False,
                'fp': False
            })

        all_relevant_classes = set(current_image_gt_boxes.keys()).union(set(detections_for_image_by_class.keys()))

        for class_name in all_relevant_classes:
            image_detections = detections_for_image_by_class[class_name]
            image_gts = current_image_gt_boxes[class_name]
            image_gts_with_flags = [{'bbox': bbox, 'detected': False} for bbox in image_gts]

            image_detections_sorted = sorted(image_detections, key=lambda x: x['score'], reverse=True)

            for det in image_detections_sorted:
                best_iou = 0.0
                best_gt_idx = -1

                for gt_idx, gt_info in enumerate(image_gts_with_flags):
                    iou = calculate_iou(det['bbox'], gt_info['bbox'])
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = gt_idx

                if best_iou >= iou_threshold_for_tp and best_gt_idx != -1 and not image_gts_with_flags[best_gt_idx]['detected']:
                    det['tp'] = True
                    image_gts_with_flags[best_gt_idx]['detected'] = True
                else:
                    det['fp'] = True

                all_detections[class_name].append((det['score'], det['tp']))

    mean_ap = 0.0
    num_classes_with_gt = 0
    ap_per_class = {}

    for class_name in PASCAL_VOC_CLASSES:
        total_gt_for_class = all_gt_counts_by_class[class_name]

        if total_gt_for_class == 0:
            ap_per_class[class_name] = 0.0
            continue

        class_detections = all_detections[class_name]
        if not class_detections:
            ap_per_class[class_name] = 0.0
            continue

        class_detections_sorted = sorted(class_detections, key=lambda x: x[0], reverse=True)

        true_positives = 0
        false_positives = 0
        precisions = []
        recalls = []

        for score, is_tp in class_detections_sorted:
            if is_tp:
                true_positives += 1
            else:
                false_positives += 1

            precision = true_positives / (true_positives + false_positives)
            recall = true_positives / total_gt_for_class

            precisions.append(precision)
            recalls.append(recall)

        ap = 0.0
        for t in np.arange(0.0, 1.1, 0.1):
            if not precisions:
                max_p = 0.0
            else:
                max_p = 0.0
                for i in range(len(recalls)):
                    if recalls[i] >= t:
                        max_p = max(max_p, precisions[i])
            ap += max_p
        ap /= 11.0

        ap_per_class[class_name] = ap
        mean_ap += ap
        num_classes_with_gt += 1

    if num_classes_with_gt > 0:
        mean_ap /= num_classes_with_gt
    else:
        mean_ap = 0.0

    log_report(f"mAP (IOU={iou_threshold_for_tp}) : {mean_ap:.4f}", level="RESULT")
    report_content_list.append(f"* mAP (IOU={iou_threshold_for_tp}) : `{mean_ap:.4f}`\n")
    log_report("AP par classe :", level="RESULT")
    report_content_list.append("\n### AP par Classe\n")
    for class_name, ap_value in ap_per_class.items():
        log_report(f"  - {class_name}: {ap_value:.4f}", level="RESULT")
        report_content_list.append(f"* {class_name}: `{ap_value:.4f}`\n")

    save_report_to_file()
    return mean_ap, ap_per_class

--- Initialisation du pipeline R-CNN ---
Heure de début de l'exécution : 2025-06-15 22:13:53
[SETUP] [22:13:53] Tentative de montage de Google Drive...
Mounted at /content/drive
[SETUP] [22:14:18] Google Drive monté avec succès. Les rapports seront enregistrés dans : /content/drive/MyDrive/RCP209/Rapports_RCNN
[INFO] [22:14:18] 
--- Vérification de l'environnement ---
[INFO] [22:14:18] PyTorch version: 2.6.0+cu124
[INFO] [22:14:18] Torchvision version: 0.21.0+cu124
[INFO] [22:14:18] OpenCV version: 4.11.0
[INFO] [22:14:18] NumPy version: 2.0.2
[INFO] [22:14:18] Scikit-learn version: 1.6.1
[INFO] [22:14:18] Un GPU est disponible : NVIDIA A100-SXM4-40GB avec 39.56 GB de mémoire.


In [ ]:
log_report("\n--- Étape 1 : Préparation de l'environnement et Téléchargement/Vérification du Dataset ---", to_console=True)

TARGET_VOC_ROOT = '/content/VOC2007_dataset'
PASCAL_VOC_ROOT = os.path.join(TARGET_VOC_ROOT, 'VOC2007')
DOWNLOAD_TEMP_ROOT = '/content/temp_voc_download'

VOC07_TRAINVAL_URL = "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar"
VOC07_TEST_URL = "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar"

def get_root_dir(output_dir=None, new_folder_name=None):
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    if not output_dir:
        root_dir = "./"
    elif os.path.isdir(output_dir):
        if new_folder_name:
            root_dir = os.path.join(output_dir, new_folder_name)
            if not os.path.exists(root_dir):
                os.mkdir(root_dir)
        else:
            root_dir = output_dir
    else:
        raise ValueError(f"{output_dir} is not a valid directory or could not be created.")
    return root_dir

def get_fname_from_path_or_url(path_or_url):
    fname_start = path_or_url.rfind("/") + 1
    fname = path_or_url[fname_start:]
    return fname

def download_one_url(url, root_dir, fname=None):
    if not fname:
        fname = get_fname_from_path_or_url(url)
    output_path = os.path.join(root_dir, fname)
    log_report(f"Downloading {fname} to {root_dir}...", level="INFO")
    with requests.get(url, stream=True, timeout=(10, 300)) as r:
        r.raise_for_status()
        with open(output_path, "wb") as f:
            total_size = int(r.headers.get('content-length', 0))
            block_size = 8192
            for chunk in r.iter_content(chunk_size=block_size):
                f.write(chunk)
    log_report(f"Downloaded {fname}.", level="INFO")
    return output_path

def unpack(archived_file_path, dest_dir):
    fname = get_fname_from_path_or_url(archived_file_path)
    log_report(f"Extracting {fname} to {dest_dir}...", level="INFO")
    shutil.unpack_archive(archived_file_path, dest_dir)
    log_report(f"Extraction of {fname} completed.", level="INFO")

def find_voc2007_root(base_path):
    for root, dirs, files in os.walk(base_path):
        if 'VOC2007' in dirs:
            voc2007_path = os.path.join(root, 'VOC2007')
            if (os.path.exists(os.path.join(voc2007_path, 'JPEGImages')) and
                os.path.exists(os.path.join(voc2007_path, 'Annotations')) and
                os.path.exists(os.path.join(voc2007_path, 'ImageSets'))):
                return voc2007_path
    return None

if not os.path.exists(os.path.join(PASCAL_VOC_ROOT, 'JPEGImages')):
    log_report("\nVOC 2007 non trouvé localement au chemin cible. Démarrage du téléchargement et de la préparation...", level="INFO")
    os.makedirs(DOWNLOAD_TEMP_ROOT, exist_ok=True)
    os.makedirs(TARGET_VOC_ROOT, exist_ok=True)

    log_report("\nPréparation du dataset d'entraînement/validation...", level="INFO")
    trainval_tar_path = download_one_url(VOC07_TRAINVAL_URL, DOWNLOAD_TEMP_ROOT)
    temp_extract_trainval_dir = os.path.join(DOWNLOAD_TEMP_ROOT, 'extracted_trainval')
    os.makedirs(temp_extract_trainval_dir, exist_ok=True)
    unpack(trainval_tar_path, temp_extract_trainval_dir)
    source_voc_trainval = find_voc2007_root(temp_extract_trainval_dir)

    if source_voc_trainval:
        if os.path.exists(PASCAL_VOC_ROOT):
            shutil.rmtree(PASCAL_VOC_ROOT)
        shutil.move(source_voc_trainval, PASCAL_VOC_ROOT)
        log_report(f"Contenu de '{source_voc_trainval}' déplacé vers '{PASCAL_VOC_ROOT}'.", level="INFO")
    else:
        log_report(f"ATTENTION: Le dossier 'VOC2007' n'a pas été trouvé dans '{temp_extract_trainval_dir}' après décompression de trainval.", level="ERROR")
        sys.exit("Structure de l'archive trainval inattendue. Arrêt du script.")

    log_report("\nPréparation du dataset de test...", level="INFO")
    test_tar_path = download_one_url(VOC07_TEST_URL, DOWNLOAD_TEMP_ROOT)
    temp_extract_test_dir = os.path.join(DOWNLOAD_TEMP_ROOT, 'extracted_test')
    os.makedirs(temp_extract_test_dir, exist_ok=True)
    unpack(test_tar_path, temp_extract_test_dir)
    source_voc_test = find_voc2007_root(temp_extract_test_dir)

    if source_voc_test:
        log_report(f"Dossier 'VOC2007' trouvé pour test à : {source_voc_test}", level="INFO")
        log_report(f"Fusion des fichiers de test vers '{PASCAL_VOC_ROOT}'.", level="INFO")
        for item in os.listdir(source_voc_test):
            src_item_path = os.path.join(source_voc_test, item)
            dest_item_path = os.path.join(PASCAL_VOC_ROOT, item)
            if os.path.isdir(src_item_path):
                shutil.copytree(src_item_path, dest_item_path, dirs_exist_ok=True)
            else:
                shutil.copy2(src_item_path, dest_item_path)
        log_report(f"Contenu de '{source_voc_test}' fusionné vers '{PASCAL_VOC_ROOT}'.", level="INFO")

    else:
        log_report(f"ATTENTION: Le dossier 'VOC2007' n'a pas été trouvé dans '{temp_extract_test_dir}' après décompression de test.", level="ERROR")
        sys.exit("Structure de l'archive test inattendue. Arrêt du script.")

    if os.path.exists(DOWNLOAD_TEMP_ROOT):
        shutil.rmtree(DOWNLOAD_TEMP_ROOT)
        log_report(f"Nettoyé le dossier temporaire : {DOWNLOAD_TEMP_ROOT}", level="INFO")

    log_report(f"VOC 2007 préparé et consolidé dans : {PASCAL_VOC_ROOT}", level="INFO")
else:
    log_report(f"\nVOC 2007 déjà préparé dans : {PASCAL_VOC_ROOT}. Skip download/extraction.", level="INFO")

report_content_list.append(f"* Chemin Dataset PASCAL VOC (consolide trainval et test) : `{PASCAL_VOC_ROOT}`\n")

[INFO] [22:14:18] 
--- Étape 1 : Préparation de l'environnement et Téléchargement/Vérification du Dataset ---
[INFO] [22:14:18] 
VOC 2007 non trouvé localement au chemin cible. Démarrage du téléchargement et de la préparation...
[INFO] [22:14:18] 
Préparation du dataset d'entraînement/validation...
[INFO] [22:14:18] Downloading VOCtrainval_06-Nov-2007.tar to /content/temp_voc_download...
[INFO] [22:14:53] Downloaded VOCtrainval_06-Nov-2007.tar.
[INFO] [22:14:53] Extracting VOCtrainval_06-Nov-2007.tar to /content/temp_voc_download/extracted_trainval...
[INFO] [22:14:55] Extraction of VOCtrainval_06-Nov-2007.tar completed.
[INFO] [22:14:55] Contenu de '/content/temp_voc_download/extracted_trainval/VOCdevkit/VOC2007' déplacé vers '/content/VOC2007_dataset/VOC2007'.
[INFO] [22:14:55] 
Préparation du dataset de test...
[INFO] [22:14:55] Downloading VOCtest_06-Nov-2007.tar to /content/temp_voc_download...
[INFO] [22:15:21] Downloaded VOCtest_06-Nov-2007.tar.
[INFO] [22:15:21] Extracting VOCt

In [ ]:
log_report("\n--- Étape 2 : Chargement et Vérification des Datasets ---", to_console=True)

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = PascalVOCDataset(root_dir=PASCAL_VOC_ROOT, year='2007', image_set='trainval', transform=image_transform)

# --- CHOIX DU DATASET DE TEST ---
DATASET_TEST_CHOICE = "VOC" # <--- Change ici pour "VOC" ou "COCO"

if DATASET_TEST_CHOICE == "VOC":
    test_dataset = PascalVOCDataset(root_dir=PASCAL_VOC_ROOT, year='2007', image_set='test', transform=image_transform)
    log_report(f"Dataset de test sélectionné : PASCAL VOC 2007 Test (taille initiale : {len(test_dataset)} images).", level="INFO")
    report_content_list.append(f"* Dataset de test utilisé : `PASCAL VOC 2007 Test`\n")
    report_content_list.append(f"* Chemin Dataset Test : `{PASCAL_VOC_ROOT}`\n")

elif DATASET_TEST_CHOICE == "COCO":
    # Ces variables viennent de la Cellule 1
    test_dataset = COCODataset(root_dir=COCO_IMAGES_VAL_PATH, annotation_file=COCO_ANNOTATIONS_VAL_PATH, transform=image_transform)
    log_report(f"Dataset de test sélectionné : MS COCO 2017 Val (taille initiale : {len(test_dataset)} images).", level="INFO")
    report_content_list.append(f"* Dataset de test utilisé : `MS COCO 2017 Val`\n")
    report_content_list.append(f"* Chemin Images COCO : `{COCO_IMAGES_VAL_PATH}`\n")
    report_content_list.append(f"* Fichier d'annotations COCO : `{COCO_ANNOTATIONS_VAL_PATH}`\n")

else:
    log_report(f"ERREUR: `DATASET_TEST_CHOICE`='{DATASET_TEST_CHOICE}' est invalide. Utilisez 'VOC' ou 'COCO'.", level="CRITICAL")
    sys.exit("Choix de dataset de test invalide. Arrêt du script.")

log_report(f"Taille du train_dataset initial : {len(train_dataset)} images.", level="INFO")
log_report(f"Taille du test_dataset initial : {len(test_dataset)} images.", level="INFO")

# --- Limitation de la taille des datasets ---
NUM_IMAGES_FOR_TRAINING = 2000 #avant 100 puis 200 maintenat 500
log_report(f"Paramètre : `NUM_IMAGES_FOR_TRAINING` = {NUM_IMAGES_FOR_TRAINING}", level="INFO")

if len(train_dataset) > NUM_IMAGES_FOR_TRAINING:
    np.random.seed(42)
    indices_train = np.random.choice(len(train_dataset), NUM_IMAGES_FOR_TRAINING, replace=False).tolist()
    train_dataset_for_use = Subset(train_dataset, indices_train)
    log_report(f"Dataset d'entraînement **limité** à {len(train_dataset_for_use)} images pour cette exécution.", level="INFO")
else:
    train_dataset_for_use = train_dataset
    log_report(f"Le dataset d'entraînement (taille {len(train_dataset)}) est déjà <= à la limite {NUM_IMAGES_FOR_TRAINING}. Aucune limitation appliquée.", level="INFO")

report_content_list.append(f"* Nombre d'images d'entraînement utilisées : `{len(train_dataset_for_use)}`\n")

NUM_IMAGES_FOR_TESTING = 100 # avant 50
log_report(f"Paramètre : `NUM_IMAGES_FOR_TESTING` = {NUM_IMAGES_FOR_TESTING}", level="INFO")

if len(test_dataset) > NUM_IMAGES_FOR_TESTING:
    np.random.seed(42)
    indices_test = np.random.choice(len(test_dataset), NUM_IMAGES_FOR_TESTING, replace=False).tolist()
    test_dataset_for_use = Subset(test_dataset, indices_test)
    log_report(f"Dataset de test **limité** à {len(test_dataset_for_use)} images pour cette exécution.", level="INFO")
else:
    test_dataset_for_use = test_dataset
    log_report(f"Le dataset de test (taille {len(test_dataset)}) est déjà <= à la limite {NUM_IMAGES_FOR_TESTING}. Aucune limitation appliquée.", level="INFO")

report_content_list.append(f"* Nombre d'images de test utilisées : `{len(test_dataset_for_use)}`\n")

log_report("\n--- Chargement des datasets terminé. ---", to_console=True)

NameError: name 'log_report' is not defined

In [ ]:
log_report("\n--- Étape 3 : Chargement du CNN (AlexNet) pré-entraîné sur ImageNet ---", to_console=True)
cnn_model = torchvision.models.alexnet(pretrained=True)
log_report("Modèle AlexNet pré-entraîné chargé. Il est actuellement sur le CPU.", level="INFO")
report_content_list.append(f"* Modèle CNN : `AlexNet (Pré-entraîné ImageNet)`\n")

log_report("\n--- Étape 4 : Fine-tuning du CNN (Cette étape utilise le GPU si activé) ---", to_console=True)
fine_tuning_start_time = time.time()

# Passez image_transform à fine_tune_cnn
fine_tuned_cnn_model = fine_tune_cnn(cnn_model, train_dataset_for_use, num_epochs=5, lr=0.001, image_transform=image_transform) # avant epoch=5 on a fait 10 on revient à 5

fine_tuning_end_time = time.time()
fine_tuning_duration = fine_tuning_end_time - fine_tuning_start_time
log_report(f"Fine-tuning du CNN terminé en {fine_tuning_duration:.2f} secondes.", level="INFO")
report_content_list.append(f"* Durée du Fine-tuning CNN : `{fine_tuning_duration:.2f} secondes`\n")

[INFO] [22:15:26] 
--- Étape 3 : Chargement du CNN (AlexNet) pré-entraîné sur ImageNet ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 225MB/s]


[INFO] [22:15:30] Modèle AlexNet pré-entraîné chargé. Il est actuellement sur le CPU.
[INFO] [22:15:30] 
--- Étape 4 : Fine-tuning du CNN (Cette étape utilise le GPU si activé) ---
[INFO] [22:15:30] Démarrage du fine-tuning du CNN pour 5 époques...
[INFO] [23:21:42] Epoch 1/5, Loss: 0.6098, Accuracy: 0.8372
[INFO] [00:27:37] Epoch 2/5, Loss: 0.4108, Accuracy: 0.8789
[INFO] [01:33:33] Epoch 3/5, Loss: 0.3311, Accuracy: 0.8994
[INFO] [02:39:40] Epoch 4/5, Loss: 0.2783, Accuracy: 0.9132
[INFO] [03:45:51] Epoch 5/5, Loss: 0.2176, Accuracy: 0.9306
[INFO] [03:45:51] Fine-tuning du CNN terminé.
[INFO] [03:45:51] Fine-tuning du CNN terminé en 19820.91 secondes.


In [ ]:
log_report("\n--- Étape 5 : Entraînement des SVMs (Cette étape utilise le GPU si activé pour l'extraction de features) ---", to_console=True)
svms_training_start_time = time.time()

# Passez image_transform à train_svms
trained_svms = train_svms(train_dataset_for_use, fine_tuned_cnn_model, image_transform=image_transform)

svms_training_end_time = time.time()
svms_training_duration = svms_training_end_time - svms_training_start_time
log_report(f"Entraînement des SVMs terminé en {svms_training_duration:.2f} secondes.", level="INFO")
report_content_list.append(f"* Durée de l'entraînement des SVMs : `{svms_training_duration:.2f} secondes`\n")

NameError: name 'log_report' is not defined

In [ ]:
log_report("\n--- Étape 6 : Démarrage de l'évaluation du détecteur (Cette étape utilise le GPU si activé) ---", to_console=True)
evaluation_start_time = time.time()

# Passez image_transform à evaluate_detector
mean_ap, ap_per_class_results = evaluate_detector(
    test_dataset_for_use,
    fine_tuned_cnn_model,
    trained_svms,
    image_transform=image_transform
)

evaluation_end_time = time.time()
evaluation_duration = evaluation_end_time - evaluation_start_time
log_report(f"Évaluation du détecteur terminée en {evaluation_duration:.2f} secondes.", level="INFO")
report_content_list.append(f"* Durée de l'évaluation du détecteur : `{evaluation_duration:.2f} secondes`\n")

# --- Fin de l'exécution globale ---
end_time_global = datetime.now()
total_duration_global = end_time_global - start_time_global
log_report(f"\n--- Exécution complète du pipeline R-CNN terminée ---", to_console=True)
log_report(f"Heure de fin : {end_time_global.strftime('%Y-%m-%d %H:%M:%S')}", to_console=True)
log_report(f"Durée totale de l'exécution : {total_duration_global}", to_console=True)

report_content_list.append(f"\n---\n\n## Résultat Final\n")
report_content_list.append(f"* Heure de fin de l'exécution : `{end_time_global.strftime('%Y-%m-%d %H:%M:%S')}`\n")
report_content_list.append(f"* Durée totale de l'exécution : `{total_duration_global}`\n")

save_report_to_file()
print("\nProcessus terminé.")